In [1]:
import calcbench as cb
from IPython.core.display import display, HTML
import json
import itertools
from tqdm import tqdm, tqdm_notebook
import random
import urllib.error
import requests
import sys
from bs4 import BeautifulSoup
import backoff
import logging
from datetime import date, timedelta
logging.getLogger('backoff').addHandler(logging.StreamHandler())
logging.getLogger('backoff').setLevel(logging.INFO)

In [ ]:
tenKQsections = cb.document_types()['tenKQSections']

In [ ]:
periods = [(year, period) for year in range(2010, 2018) for period in [0, 1, 2, 3]]

In [ ]:
missing_docs = []
def get_documents(disclosure_name, year, period, company_identifiers=[]):
    total_length = 0
    text_length = 0
    entire_universe = not company_identifiers
    try:
        found_documents = doc_search(company_identifiers=company_identifiers,
                                      entire_universe=entire_universe, 
                                      document_name=disclosure_name, 
                                      year=year, 
                                      period=period)
    except requests.RequestException:
        print(disclosure_type, year, period)
    else:
        contents_length = 0
        docs = []
        for  doc in tqdm_notebook(found_documents, desc="{0} {1}-{2}".format(disclosure_name, year, period), leave=False):
            try:
                contents = get_doc_contents(doc)
                total_length = total_length + len(contents)
                text_length += len(BeautifulSoup(contents, 'html.parser').text)
            except requests.RequestException:
                missing_docs.append(doc)
            except json.JSONDecodeError:
                print(disclosure_name, year, period, doc['ticker'])
    return total_length, text_length

In [ ]:
@backoff.on_exception(backoff.expo, (requests.exceptions.RequestException), 8)
def doc_search(*args, **kwargs):
    for r in tqdm_notebook(cb.document_search(*args, **kwargs), desc="search results"):
        yield r

In [ ]:
@backoff.on_exception(backoff.expo, (requests.exceptions.RequestException), 8)
def get_doc_contents(doc):
    return doc.get_contents()

In [ ]:
sample_sections = [d['name']  for d in cb.document_types()['disclosures']]

In [ ]:
test_doc_periods = list(itertools.product(sample_sections, periods))

In [ ]:
total_length = 0
for test_doc, period in tqdm_notebook(test_doc_periods, desc="period loop"):
    total_length, text_length = get_documents(test_doc, year=period[0], period=period[1])

In [4]:
all_10_K_Q_sections = cb.document_types()['tenKQSections']
yesterday = date.today() - timedelta(1)
sections_from_document_recieved_since_yesterday = []
for section in all_10_K_Q_sections:
    print(section)
    print(len(list(cb.document_search(updated_from=yesterday, entire_universe=True, document_name=section['name']))))

{'arcrole': 1100, 'name': 'Business Description', 'shortName': 'Business Description', 'tags': None, 'documentType': '10-K/Q Disclosures', 'longName': 'Business Description'}
11
{'arcrole': 1110, 'name': 'Risk Factors', 'shortName': 'Risk Factors', 'tags': None, 'documentType': '10-K/Q Disclosures', 'longName': 'Risk Factors'}
579
{'arcrole': 1120, 'name': 'Unresolved SEC Comments', 'shortName': 'Unresolved Comments', 'tags': None, 'documentType': '10-K/Q Disclosures', 'longName': 'Unresolved SEC Comments'}
1
{'arcrole': 1200, 'name': 'Properties', 'shortName': 'Properties', 'tags': None, 'documentType': '10-K/Q Disclosures', 'longName': 'Properties'}
10
{'arcrole': 1300, 'name': 'Legal Proceedings', 'shortName': 'Legal Proceedings', 'tags': None, 'documentType': '10-K/Q Disclosures', 'longName': 'Legal Proceedings'}
445
{'arcrole': 2410, 'name': 'Executive Officers', 'shortName': 'Executive Officers', 'tags': None, 'documentType': '10-K/Q Disclosures', 'longName': 'Executive Officers'

In [2]:
cb.api_client._rig_for_testing('localhost')